In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [40]:
pd.set_option("display.max_columns", 100)

In [3]:
analysis_date = '2022-05-25'
itp_id = 300


In [4]:
# query_sql(f'''
# SELECT * FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
# ''')

In [17]:
updates = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
WHERE date = '{analysis_date}' and calitp_itp_id = {itp_id}
LIMIT 1000
''')

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [18]:
updates.head()

,calitp_itp_id,calitp_url_number,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key
0,300,0,gtfs-data/rt/2022-05-25T20:00:07/300/0/gtfs_rt...,2022-05-25,883322_block_9320_schedBasedVehicle_52380,1653508751,None,block_9320_schedBasedVehicle,None,None,883322,3495,0,14:33:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '38', 'arrival'...",ad198acfe666682db6ca84cf82a4cb29
1,300,0,gtfs-data/rt/2022-05-25T20:00:07/300/0/gtfs_rt...,2022-05-25,880860_1818_43200,1653508566,None,1818,None,None,880860,3479,1,12:00:00,20220525,SCHEDULED,"[{'stopSequence': 45, 'stopId': '1305', 'arriv...",2fac8226a311da09b635ef5bcc2a4b92
2,300,0,gtfs-data/rt/2022-05-25T20:00:07/300/0/gtfs_rt...,2022-05-25,881531_1356_43800,1653508632,None,1356,None,None,881531,3483,0,12:10:00,20220525,SCHEDULED,"[{'stopSequence': 48, 'stopId': '1354', 'arriv...",d45678e166b53e33f24c7d62dd394788
3,300,0,gtfs-data/rt/2022-05-25T20:00:07/300/0/gtfs_rt...,2022-05-25,882511_1332_45360,1653508650,None,1332,None,None,882511,3489,0,12:36:00,20220525,SCHEDULED,"[{'stopSequence': 14, 'stopId': '1184', 'arriv...",b15b3be5f408584ffa47d55d6efae0b4
4,300,0,gtfs-data/rt/2022-05-25T20:00:27/300/0/gtfs_rt...,2022-05-25,883322_block_9320_schedBasedVehicle_52380,1653508811,None,block_9320_schedBasedVehicle,None,None,883322,3495,0,14:33:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '38', 'arrival'...",9447d236ae61a0c87d8a8ed045fc10a6


In [19]:
updates.trip_schedule_relationship.value_counts()

SCHEDULED    1000
Name: trip_schedule_relationship, dtype: int64

In [20]:
updates.trip_id.value_counts()

882517    7
881488    7
881839    7
882897    7
881836    7
         ..
881831    2
880762    2
880848    2
880860    1
881531    1
Name: trip_id, Length: 169, dtype: int64

In [21]:
updates.trip_direction_id.value_counts()

0    525
1    475
Name: trip_direction_id, dtype: int64

In [22]:
updates.vehicle_id.value_counts()

block_4303_schedBasedVehicle    24
1821                            19
1820                            19
1824                            18
1319                            18
                                ..
2903                             6
2105                             6
1505                             6
1816                             6
1716                             2
Name: vehicle_id, Length: 92, dtype: int64

In [23]:
updates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   calitp_itp_id               1000 non-null   int64 
 1   calitp_url_number           1000 non-null   int64 
 2   original_file_path          1000 non-null   object
 3   date                        1000 non-null   object
 4   id                          1000 non-null   object
 5   timestamp                   1000 non-null   int64 
 6   delay                       0 non-null      object
 7   vehicle_id                  1000 non-null   object
 8   vehicle_label               0 non-null      object
 9   vehicle_license_plate       0 non-null      object
 10  trip_id                     1000 non-null   object
 11  trip_route_id               1000 non-null   object
 12  trip_direction_id           1000 non-null   int64 
 13  trip_start_time             1000 non-null   objec

In [13]:
## adding in scheduled and ran trips: 
def find_pct_ran_trips(itp_id, analysis_date):
    gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter(_.service_date == analysis_date)
        >> filter(_.is_in_service == True)
        >> collect()
    )
    rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.nla_views.test_rt_trips`
        WHERE date = '{analysis_date}'
        """
    )
    
    rt['str_len'] = rt.trip_id.str.len()
    
    #joinging 
    join = (pd.merge(gtfs_daily, rt, how='outer', on='trip_id', indicator='have_rt'))
    
    return join

In [14]:
trips = find_pct_ran_trips(itp_id, analysis_date)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [29]:
len(trips)

1618

In [16]:
trips.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,...,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,str_len,have_rt
0,4100255323842349957,-276168916209754772,882890,3491,300,0,10,2022-05-25,1,2022-03-27,...,16,24240,25140,0.250000,300.0,0.0,2022-05-25,1820,6.0,both
1,4100255323842349957,5450854513072761642,883706,3501,300,0,40510,2022-05-25,0,2022-03-27,...,17,58560,59760,0.333333,NaN,NaN,NaN,NaN,NaN,left_only
2,4100255323842349957,4962984872484144342,882619,3489,300,0,10,2022-05-25,1,2022-03-27,...,17,61380,63540,0.600000,300.0,0.0,2022-05-25,1824,6.0,both
3,4100255323842349957,4962984872484144342,882619,3489,300,0,10,2022-05-25,1,2022-03-27,...,17,61380,63540,0.600000,300.0,0.0,2022-05-25,2106,6.0,both
4,4100255323842349957,-3228323852825060608,882569,3489,300,0,10,2022-05-25,1,2022-03-27,...,17,25800,27240,0.400000,300.0,0.0,2022-05-25,2114,6.0,both


In [30]:
len(updates)

1000

In [25]:
updates.head()

,calitp_itp_id,calitp_url_number,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key
0,300,0,gtfs-data/rt/2022-05-25T20:00:07/300/0/gtfs_rt...,2022-05-25,883322_block_9320_schedBasedVehicle_52380,1653508751,None,block_9320_schedBasedVehicle,None,None,883322,3495,0,14:33:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '38', 'arrival'...",ad198acfe666682db6ca84cf82a4cb29
1,300,0,gtfs-data/rt/2022-05-25T20:00:07/300/0/gtfs_rt...,2022-05-25,880860_1818_43200,1653508566,None,1818,None,None,880860,3479,1,12:00:00,20220525,SCHEDULED,"[{'stopSequence': 45, 'stopId': '1305', 'arriv...",2fac8226a311da09b635ef5bcc2a4b92
2,300,0,gtfs-data/rt/2022-05-25T20:00:07/300/0/gtfs_rt...,2022-05-25,881531_1356_43800,1653508632,None,1356,None,None,881531,3483,0,12:10:00,20220525,SCHEDULED,"[{'stopSequence': 48, 'stopId': '1354', 'arriv...",d45678e166b53e33f24c7d62dd394788
3,300,0,gtfs-data/rt/2022-05-25T20:00:07/300/0/gtfs_rt...,2022-05-25,882511_1332_45360,1653508650,None,1332,None,None,882511,3489,0,12:36:00,20220525,SCHEDULED,"[{'stopSequence': 14, 'stopId': '1184', 'arriv...",b15b3be5f408584ffa47d55d6efae0b4
4,300,0,gtfs-data/rt/2022-05-25T20:00:27/300/0/gtfs_rt...,2022-05-25,883322_block_9320_schedBasedVehicle_52380,1653508811,None,block_9320_schedBasedVehicle,None,None,883322,3495,0,14:33:00,20220525,SCHEDULED,"[{'stopSequence': 1, 'stopId': '38', 'arrival'...",9447d236ae61a0c87d8a8ed045fc10a6


In [26]:
## joining updates and trips

In [32]:
join = (pd.merge(trips, updates, how='outer', on='trip_id', indicator='have_updates'))


In [43]:
join.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date_x,vehicle_id_x,str_len,have_rt,calitp_itp_id,calitp_url_number,original_file_path,date_y,id,timestamp,delay,vehicle_id_y,vehicle_label,vehicle_license_plate,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key,have_updates
0,4100255323842349957,-276168916209754772,882890,3491,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,16,16,24240,25140,0.250000,300.0,0.0,2022-05-25,1820,6.0,both,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,4100255323842349957,5450854513072761642,883706,3501,300,0,40510,2022-05-25,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,17,17,58560,59760,0.333333,NaN,NaN,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,4100255323842349957,4962984872484144342,882619,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,61380,63540,0.600000,300.0,0.0,2022-05-25,1824,6.0,both,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,4100255323842349957,4962984872484144342,882619,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,61380,63540,0.600000,300.0,0.0,2022-05-25,2106,6.0,both,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,4100255323842349957,-3228323852825060608,882569,3489,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17,17,25800,27240,0.400000,300.0,0.0,2022-05-25,2114,6.0,both,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [33]:
join.have_updates.value_counts()

left_only     1448
both          1006
right_only       0
Name: have_updates, dtype: int64

In [42]:
(join>>filter(_.have_updates=='left_only')).have_rt.value_counts()

both          1280
left_only      168
right_only       0
Name: have_rt, dtype: int64

In [45]:
join>>filter(_.have_updates=='both')>>count(_.trip_id)

,trip_id,n
0,880762,2
1,880763,6
2,880764,7
3,880765,6
4,880766,6
...,...,...
164,883554,6
165,883574,6
166,883581,6
167,883599,6
